
# 迁移学习
大型神经网络的训练困难
训练大型模型时，用于降低数据需求以及加快训练速度的关键技术——预训练pre-train （迁移学习transfer learning）

借用已经训练好的模型来构筑新架构


- 基于经典架构构建自己的架构时：
    - 将经典架构本身复制，在前后增加我们希望的层
    - 这个过程中的经典架构并没有被训练过，全部层在训练时都得初始化自己的参数，从0开始训练
- 迁移学习
    - 复用已经训练好的架构，包括架构本身以及每层上的权重
    - 沿用现存架构以及其权重后，在后面加入自定义的层，以此来构筑新的架构

隐藏层-信息储存在权重中
- 网络浅层：提取浅层信息（常识）
- 网络深层：提取神经信息（具体信息）

可以将浅层信息迁移过去，迁移对象
- `任务相似`
- 数据高度相似

训练时的两种选择：
1. 将迁移层上的权重作为初始化工具

    将迁移层的权重作为新架构的初始化权重，在此基础上对所有层进行训练，给模型指引方向。严谨文献中称为“预训练”
2. 将迁移层作为固定的特征提取工具

    将迁移层的权重“固定”起来，不让这些权重收到反向传播等过程的影响，而让他们作为架构中的“固定知识”被一直使用。相对的，自定义添加的层像普通的网络架构一样初始化参数并进行训练，并在每一次迭代中逐渐找到适合自己的权重。严谨文献中称为“迁移学习”

可以在刚开始训练时固定所有迁移层的权重，大步长迭代学习下面的隐藏层，等到模型结果逐渐不再持续上升时，尝试解锁一两个层并使用小的学习率。

In [1]:
import torch
import torch.nn as nn
from torchvision import models as m

resnet18_ = m.resnet18()

# 执行该代码时要关闭VPN
# 下载预训练好的权重
rs18pt = m.resnet18(pretrained=True) # resnet18_pretrained

In [2]:
resnet18_.conv1.weight[0] # 初始化的参数，准备好预训练

tensor([[[ 0.0149, -0.0052, -0.0291,  0.0267, -0.0076, -0.0162, -0.0188],
         [ 0.0222, -0.0382,  0.0143, -0.0383, -0.0120, -0.0095,  0.0253],
         [ 0.0099, -0.0146, -0.0072,  0.0347,  0.0028, -0.0095, -0.0026],
         [ 0.0011, -0.0047, -0.0086,  0.0197,  0.0021, -0.0307,  0.0407],
         [-0.0123, -0.0054,  0.0066,  0.0367,  0.0212, -0.0253,  0.0039],
         [ 0.0588, -0.0258, -0.0818, -0.0036,  0.0236, -0.0223,  0.0247],
         [-0.0053, -0.0195, -0.0177,  0.0176, -0.0011,  0.0129, -0.0116]],

        [[-0.0105,  0.0224, -0.0290, -0.0490, -0.0088, -0.0114, -0.0399],
         [-0.0044,  0.0405,  0.0116, -0.0110,  0.0330,  0.0023,  0.0056],
         [ 0.0533,  0.0197, -0.0495,  0.0005,  0.0282, -0.0240, -0.0001],
         [-0.0111, -0.0017,  0.0689, -0.0112,  0.0120, -0.0182, -0.0368],
         [-0.0017, -0.0033,  0.0054,  0.0020,  0.0035, -0.0249, -0.0172],
         [-0.0376,  0.0592,  0.0334,  0.0122, -0.0030, -0.0047, -0.0184],
         [ 0.0032,  0.0082, -0.0124,

In [3]:
rs18pt.conv1.weight[0] # 经过预训练的参数
# grad_fn=<SelectBackward> 可以被反向传播

tensor([[[-1.0419e-02, -6.1356e-03, -1.8098e-03,  7.4841e-02,  5.6615e-02,
           1.7083e-02, -1.2694e-02],
         [ 1.1083e-02,  9.5276e-03, -1.0993e-01, -2.8050e-01, -2.7124e-01,
          -1.2907e-01,  3.7424e-03],
         [-6.9434e-03,  5.9089e-02,  2.9548e-01,  5.8720e-01,  5.1972e-01,
           2.5632e-01,  6.3573e-02],
         [ 3.0505e-02, -6.7018e-02, -2.9841e-01, -4.3868e-01, -2.7085e-01,
          -6.1282e-04,  5.7602e-02],
         [-2.7535e-02,  1.6045e-02,  7.2595e-02, -5.4102e-02, -3.3285e-01,
          -4.2058e-01, -2.5781e-01],
         [ 3.0613e-02,  4.0960e-02,  6.2850e-02,  2.3897e-01,  4.1384e-01,
           3.9359e-01,  1.6606e-01],
         [-1.3736e-02, -3.6746e-03, -2.4084e-02, -6.5877e-02, -1.5070e-01,
          -8.2230e-02, -5.7828e-03]],

        [[-1.1397e-02, -2.6619e-02, -3.4641e-02,  3.6812e-02,  3.2521e-02,
           6.6221e-04, -2.5743e-02],
         [ 4.5687e-02,  3.3603e-02, -1.0453e-01, -3.0885e-01, -3.1253e-01,
          -1.6051e-01, -1.2

In [4]:
# 属性 requires_grad 为 True，意味着可以参与反向传播
# 预训练的参数刚被导入时，都是默认可以被训练的
rs18pt.conv1.weight[0].requires_grad

True

In [5]:
rs18pt.parameters() # 类似DataLoader，导入后是生成器

<generator object Module.parameters at 0x7f8a04368660>

In [6]:
# 将导入的预训练模型中所有的参数锁住
for param in rs18pt.parameters():
    param.requires_grad = False

In [7]:
rs18pt.conv1.weight[0][0]
# grad_fn=<SelectBackward> 属性消失，意味着这些参数不能参与反向传播等训练流程了

tensor([[-0.0104, -0.0061, -0.0018,  0.0748,  0.0566,  0.0171, -0.0127],
        [ 0.0111,  0.0095, -0.1099, -0.2805, -0.2712, -0.1291,  0.0037],
        [-0.0069,  0.0591,  0.2955,  0.5872,  0.5197,  0.2563,  0.0636],
        [ 0.0305, -0.0670, -0.2984, -0.4387, -0.2709, -0.0006,  0.0576],
        [-0.0275,  0.0160,  0.0726, -0.0541, -0.3328, -0.4206, -0.2578],
        [ 0.0306,  0.0410,  0.0628,  0.2390,  0.4138,  0.3936,  0.1661],
        [-0.0137, -0.0037, -0.0241, -0.0659, -0.1507, -0.0822, -0.0058]])

In [8]:
# 同时 requires_grad 属性变为 False
rs18pt.conv1.weight[0].requires_grad

False

In [9]:
# 使用新层覆盖原来的层
rs18pt.conv1 = nn.Conv2d(1,64,kernel_size=(7,7),stride=(2,2),padding=(3,3),bias=False)

# 新生成的层默认 requires_grad=True
# 因此在锁定模型中的参数后，只要覆盖掉原来的层，或者在原来的层之后加上新层，新层默认就是可以训练的
# 但是新的层会覆盖掉原来层已经训练好的参数，所以一般不对conv1进行覆盖
rs18pt.conv1.weight.requires_grad

True

In [10]:
# 按照该逻辑定义架构
# 让18层残差网络的前两个 layers 都被冻结，后面两个 layers 从0开始训练
resnet18_ = m.resnet18() # 没有预训练的模型
rs18pt = m.resnet18(pretrained=True) # resnet18_pretrained

for param in rs18pt.parameters():
    param.requires_grad = False

fcin = rs18pt.fc.in_features

class MyNet_pretrained(nn.Module):
    def __init__(self):
        super().__init__()
        # 迁移层
        self.pretrained = nn.Sequential(rs18pt.conv1,
                                        rs18pt.bn1,
                                        rs18pt.relu,
                                        rs18pt.maxpool,
                                        rs18pt.layer1,
                                        rs18pt.layer2
                                       )
        # 允许训练的层
        self.train_ = nn.Sequential(resnet18_.layer3
                                   ,resnet18_.layer4
                                   ,resnet18_.avgpool)
        # 输出的线性层自己写，以确保输出的类别数量正确
        self.fc = nn.Linear(in_features=fcin,out_features=10,bias=True)
        
    def forward(self,x):
        x = self.pretrained(x)
        x = self.train_(x)
        x = x.view(x.shape[0],512)
        x = self.fc(x)
        
        return x

In [11]:
net = MyNet_pretrained()

net.pretrained[0].weight.requires_grad

False

In [12]:
net.train_[0][0].conv1.weight.requires_grad

True

In [13]:
from torchinfo import summary
summary(net,input_size=(10,3,224,224),depth=3,device="cpu")
# 输出中，不能训练的参数被括号括了起来，表示被锁定

Layer (type:depth-idx)                        Output Shape              Param #
MyNet_pretrained                              --                        --
├─Sequential: 1-1                             [10, 128, 28, 28]         --
│    └─Conv2d: 2-1                            [10, 64, 112, 112]        (9,408)
│    └─BatchNorm2d: 2-2                       [10, 64, 112, 112]        (128)
│    └─ReLU: 2-3                              [10, 64, 112, 112]        --
│    └─MaxPool2d: 2-4                         [10, 64, 56, 56]          --
│    └─Sequential: 2-5                        [10, 64, 56, 56]          --
│    │    └─BasicBlock: 3-1                   [10, 64, 56, 56]          (73,984)
│    │    └─BasicBlock: 3-2                   [10, 64, 56, 56]          (73,984)
│    └─Sequential: 2-6                        [10, 128, 28, 28]         --
│    │    └─BasicBlock: 3-3                   [10, 128, 28, 28]         (230,144)
│    │    └─BasicBlock: 3-4                   [10, 128, 28, 28]     

In [14]:
# 训练一段时间后，希望解锁部分层

In [15]:
net.pretrained # 提取前面锁住的部分

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [16]:
net.pretrained[5] # 第五个块

Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (downsample): Sequential(
      (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): BasicBlock(
    (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(128, eps=1e-

In [17]:
net.pretrained[5][1] # 第五个块中的第二个残差块

BasicBlock(
  (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [18]:
net.pretrained[5][1].parameters() # 该层中的所有参数

<generator object Module.parameters at 0x7f89ea15a7b0>

In [19]:
# 解锁被锁定部分的最后一个layers
for param in net.pretrained[5][1].parameters():
    param.requires_grad = True

In [20]:
net.pretrained[5][1].conv1.weight.requires_grad

True

In [21]:
from torchinfo import summary
summary(net,input_size=(10,3,224,224),depth=3,device="cpu")
# 输出中，不能训练的参数被括号括了起来，表示被锁定

Layer (type:depth-idx)                        Output Shape              Param #
MyNet_pretrained                              --                        --
├─Sequential: 1-1                             [10, 128, 28, 28]         --
│    └─Conv2d: 2-1                            [10, 64, 112, 112]        (9,408)
│    └─BatchNorm2d: 2-2                       [10, 64, 112, 112]        (128)
│    └─ReLU: 2-3                              [10, 64, 112, 112]        --
│    └─MaxPool2d: 2-4                         [10, 64, 56, 56]          --
│    └─Sequential: 2-5                        [10, 64, 56, 56]          --
│    │    └─BasicBlock: 3-1                   [10, 64, 56, 56]          (73,984)
│    │    └─BasicBlock: 3-2                   [10, 64, 56, 56]          (73,984)
│    └─Sequential: 2-6                        [10, 128, 28, 28]         --
│    │    └─BasicBlock: 3-3                   [10, 128, 28, 28]         (230,144)
│    │    └─BasicBlock: 3-4                   [10, 128, 28, 28]     

In [22]:
# 在 PyTorch 中我们可以非常灵活的调用任何层或层上的参数
# 这为我们灵活调用层来组成自己希望的预训练模型提供较好的基础

In [23]:
# 来源：github / 实验室 获得的模型或权重
# 格式：pt,pth
# 否则需要先将模型文件转化为 pytorch 可以读取的类型

In [ ]:
#========【从url获取模型权重】=========
url = 'http://xxx/xxx.pth'

# 定义 model 架构，并实例化 model,model架构必须与 url 中权重要求的架构一模一样
model = TheModelClass(*args,**kwargs)

# state_dict 模型的完整形式，包括结构和参数
state_dict = load_state_dict_from_url(url)
model.load_state_dict(state_dict)

In [24]:
net.state_dict

<bound method Module.state_dict of MyNet_pretrained(
  (pretrained): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momen

In [ ]:
#========【从保存好的权重文件中获取模型权重】=========
PATH = 'xxx/xxx.pth'

# 实例化模型
model = TheModelClass(*args,**kwargs)

model.load_state_dict(torch.load(PATH))

In [ ]:
#========【从保存好的模型中获取模型权重】=========
PATH = 'xxx/xxx.pth'

model = torch.load(PATH)

# 获取权重
model.state_dict()
best_model_wts = copy.deepcopy(model.state_dict()) # 深拷贝

In [ ]:
# 选择对 state_dict() 中的部分值进行迭代
model.load_state_dict(best_model_wts)